# Video Object Detection and Tracking with YOLOv3 (OpenCV library)

In this notebook, we will show step by step how to detect and track objects in a video (i.e. a real-time video from a highway camera).   

## Importing Libraries

First step, we need to import OpenCV, Numpy, Matplotlib libraries and Centroid Tracker.

In [1]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video
from centroidtracker import *

## Video

Let's have a look at the video.

Link (click on "download" and watch it):   [https://github.com/buropas/Object_Detection/blob/main/test.mp4](https://github.com/buropas/Object_Detection/blob/main/test.mp4)

In [2]:
#Take a look at the input video
Video("test.mp4")

## Loading Model Configuration and Pre-trained Weights

Yolov3 pre-trained weights can be downloaded here: https://pjreddie.com/media/files/yolov3.weights.

Let's load Yolov3 model configuration and pre-trained weights by reading "yolov3.cfg" and yolov3.weights".
We also need to extract class names from the file "coco.names".

In [3]:
## Loading network configuration and pre-trained weights 
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg") 

## Save all the class names in a list  (80 CLASSES)
with open("coco.names", "r") as f:     
    classes = [word.strip() for word in f.readlines()] 
    
## Get layer names of the network 
layer_names = net.getLayerNames() 

## Determine the output layer names from the YOLO model  
# (net.getUnconnectedOutLayers() gives the index position of the layers)
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()] 

print("YOLOv3 LOADED SUCCESSFULLY")

YOLOv3 LOADED SUCCESSFULLY


This is all we need in order to load the model configuration and pre-trained weights.  

## Capturing video 
Next step is to load the video and define the VideoWriter object in order to save our final video with detected objects. The output video will be saved as "output.avi".

In the VideoWriter object we specify:
- the output file name (output.avi), 
- the FourCC code (a 4-byte code used to specify the video codec), 
- the number of frames per second (fps), 
- the frame size.

In [79]:
## Loading video 
filename = "test.mp4"                         # filename
cap = cv2.VideoCapture(filename)              # loading video

# We get the resolution of our video (width and height) and we convert from float to integer
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# We create VideoWriter object and define the codec. The output is stored in 'output.avi' file.
out_video = cv2.VideoWriter("output_track.avi",                                # output name
                            cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'),  # 4-byte code used to specify the video codec
                                                                         # (we pass MJPG)
                            15,                                          # number of frames per second (fps) 
                            (frame_width, frame_height)                  # frame size
                            )

# set font and color of text and bounding boxes
font = cv2.FONT_HERSHEY_PLAIN     # font
color_g = (0, 255, 0)             # green color
color_r = (0, 0, 255)             # red color

## Object Detection + Object Tracking

The goal is to perform object detection and tracking only in a specific region (RoI - Region of Interest) of the video: we want to detect and track the incoming vehicles in the right part of the video.     
So, the idea is that we process the video frame by frame.   
For each frame:
- we select the region of interest, preprocess it and pass it as input into the network,
- we obtain detected objects in the region of interest as output of our network,
- we discard objects detected with a confidence score lower than a specific threshold and we perform Non-maximum Suppression (NMS), which is a technique to filter the predictions,
- we show detected objects with bounding boxes and classes,
- furthermore, we track each detected object using a Centroid-based tracking algorithm.   
 The algorithm assigns a unique ID to the detected object and stores the centroid of the bounding box coordinates for that object.   
 Then, the algorithm updates IDs list when new vehicles are detected or old ones are removed and updates centroid coordinates as the vehicles move around frames. 

At the end, we save the video with detected and tracked objects.

##  Centroid-based Tracking

Given an initial set of bounding box coordinates for each detected objects, Object Tracking consists in assigning a unique ID for each of the initial detections and then tracking each of the objects as they move around frames in a video, maintaining the assignment of unique IDs.

In this project, we perform Object Tracking using a Centroid-based tracking algorithm.

The Centroid Tracking algorithm requires a set of bounding box produced by the object detector for each detected object in every single frame.

Once we have the bounding box coordinates we must compute the “centroid”, i.e. the x, y coordinates of the center point in the bounding box.


The Centroid Tracking algorithm is a tracking algorithm  that relies on the Euclidean distance between existing object centroids (i.e., objects the centroid tracker has already seen before) and new object centroids between subsequent frames in a video.

The "maxDisappeared" hyperparameter refers to the number of consecutive frames an object is allowed to be marked as “lost/disappeared” until the object is deregistered.   In the case that an object leaves the frame and does not come back for "maxDisappeared" frames, a new object ID would be assigned.

In [80]:
# Instantiate CentroidTracker for object tracking 
ct = CentroidTracker(maxDisappeared=8)      # max 8 frames for the object to be lost without being deregistered

In [81]:
while cap.isOpened():     # while the capture is correctly initialized...

    # We process the video frame-by-frame
    
    ret, img = cap.read()           # we read each frame (img) from the video
                                    # we also retrieve ret, which is a boolean value. 
                                    # ret is True if the frame is read correctly
    
    if ret == True:    # if the frame is read correctly, go on...
        
    
        ## EXTRACT REGION OF INTEREST(ROI)
        roi = img[210:, 450:]           # consider only a slice in pixels of the entire frame 

        height, width, _ = roi.shape    # retrieve height and width from the region of interest
                                        # (we need height and width to build bounding boxes later)

        ## IMAGE PREPROCESSING
        # The cv2.dnn.blobFromImage function returns a blob which is our input image after
        # scaling by a scale factor, and channel swapping.
        # The input image that we need to pass to the Yolo algorithm must be 416x416
        blob = cv2.dnn.blobFromImage(roi, 1/255.0, (416,416), (0,0,0), swapRB=True, crop= False)

        ######################
        ## OBJECT DETECTION ##
        ######################
        
        net.setInput(blob)                      # set blob as input to the network
        outs = net.forward(output_layers)       # runs a forward pass to compute the network output  


        boxes = []
        confidences = []
        class_ids = []

        for out in outs:            # for each output...
            for detection in out:               # for each detection...
                scores = detection[5:]             # array with 80 scores (1 score for each class)
                class_id = np.argmax(scores)       # take the id of the maximum score
                confidence = scores[class_id]      # confidence of the class with the maximum score

                if confidence > 0.5:    # if the confidence of the detected object is above the threshold
                                        # we start to create the bounding box...
                    # Object detected
                    center_x = int(detection[0] * width)             # x of the center point
                    center_y = int(detection[1] * height)            # y of the center point
                    w = int(detection[2] * width)                    # width of the detected object
                    h = int(detection[3] * height)                   # height of the detected object

                    # Rectangle coordinates
                    x = int(center_x - w / 2)                         # x of the top left point
                    y = int(center_y - h / 2)                         # y of the top left point

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)


        ## NMS - NON-MAXIMUM SUPPRESSION
        # We use NMS function in opencv to perform Non-maximum Suppression  
        # The function performs non maximum suppression, given boxes and corresponding confidence scores
        # We give it score threshold and nms threshold as arguments:
        # score_threshold: keep only boxes with a confidence score higher than the threshold
        # nms threshold: threshold used in non maximum suppression (IoU)
        # The function returns indices of bounding boxes survived after NMS.
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold = 0.5, nms_threshold = 0.3)
        
        
        ## Draw bounding boxes of the final detected objects 
        final_boxes = []
        for i in range(len(boxes)):              # for each box...
            if i in indexes:                     # if the bounding box has survived after NMS...  
        
                x, y, w, h = boxes[i]                                      # bounding box coordinates
                label = str(classes[class_ids[i]])                         # class label
                cv2.rectangle(roi, (x, y), (x + w, y + h), color_g, 2)     # drawing rectangular bounding box
                                                                # (x,y) is the top left corner of the box
                                                                # (x + w, y + h) is the bottom right corner of the box
                cv2.putText(roi, label, (x, y - 5), font, 1, color_r, 2)   # class of the detected object 
                final_boxes.append([x, y, x+w, y+h])            # append bounding box to the final boxes list

        
        #####################
        ## OBJECT TRACKING ##
        #####################
        
        objects = ct.update(final_boxes)   # we pass the list with final boxes to the tracker in order to have 
                                           # a unique id for each detection and its specific centroid coordinates

        ## Draw the ID of the detected and tracked object
        for (objectID, centroid) in objects.items():    # for each tracked object...
            text = str(objectID +1)                                # unique id of the tracked object
            cv2.putText(roi, text, (centroid[0], centroid[1] -30), font, 2, color_g, 2),   # text
            #cv2.circle(roi, (centroid[0], centroid[1]), 4, (0, 255, 0), -1) # draw the centroid of tracked object
            
    
        cv2.imshow("out", roi)      # display the current frame with detected objects  

        out_video.write(img)        # the frame is saved for the final video

        key = cv2.waitKey(1)        # wait 1 millisecond between each frame
        if key == 27:               # if exit button, break and close
            break
    
    
    else:   # if the frame is not read correctly, break...
        break
    
# Release everything when job is finished
cap.release()
out_video.release()
cv2.destroyAllWindows()

Now, let's have a look at the final result...

## Video with Detected and Tracked Objects

Link to Output Video (click on "download" and just watch or download it):   

[https://github.com/buropas/Object_Detection/blob/main/output_track.avi](https://github.com/buropas/Object_Detection/blob/main/output_track.avi)